In [1]:
print("This is a test")

This is a test


In [2]:
from hogwarts.auth.vault.vault_client import VaultClient

vault_client = VaultClient()
vault_client.login()

test = vault_client.get_personal_secret("Test-Secret")

testUser = test["test_username"]
testValue = test["test_value"]

print("Testing secrets")
print("Username: " + testUser)
print("Value: " + testValue)

print("g_sqlDatabase: " + g_sqlDatabase)


Testing secrets
Username: test.test
Value: 12345
